In [1]:
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [2]:
device = 'cuda'

In [3]:
x = np.load('./xtrain.npy')
y = np.load('./ytrain.npy')

In [4]:
x = torch.tensor(x,dtype=torch.float32).to(device)
y = torch.tensor(y,dtype=torch.float32).to(device)

In [8]:
# Create a dataset and dataloader
dataset = TensorDataset(x, y)
dataloader = DataLoader(dataset, batch_size=512, shuffle=True)

In [9]:
# Define the MLP equivalent to sklearn's MLPRegressor
class MLPRegressorPyTorch(nn.Module):
    def __init__(self):
        super(MLPRegressorPyTorch, self).__init__()
        self.fc1 = nn.Linear(1800, 3600)  # Assuming input features are 10
        self.fc2 = nn.Linear(3600, 7200)
        self.fc3 = nn.Linear(7200, 14400)
        self.fc4 = nn.Linear(14400, 7200)
        self.fc5 = nn.Linear(7200, 3600)
        self.out = nn.Linear(3600, 1800)   # Output layer

        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.relu(self.fc4(x))
        x = self.relu(self.fc5(x))
        x = self.out(x)  # No activation in the output layer for regression
        return x

In [12]:
# Initialize model, loss function, and optimizer
model = MLPRegressorPyTorch().to(device)
criterion = nn.MSELoss()  # Mean Squared Error Loss (similar to MLPRegressor)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [13]:
# Training loop
num_epochs = 100  # You can adjust the number of epochs
loss_min = 1e8
for epoch in range(num_epochs):
    total_loss = 0
    for batch_x, batch_y in dataloader:
        # Forward pass
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        if(loss.item()<loss_min):
            torch.save(model.state_dict(), './model_ds1.pth')
            loss_min = loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(dataloader):.4f}')

print("Training complete.")

Epoch [1/100], Loss: 407.0677
Epoch [2/100], Loss: 21.4492
Epoch [3/100], Loss: 18.0115
Epoch [4/100], Loss: 17.7517
Epoch [5/100], Loss: 17.7235
Epoch [6/100], Loss: 17.7195
Epoch [7/100], Loss: 17.7209
Epoch [8/100], Loss: 17.7225
Epoch [9/100], Loss: 17.7235
Epoch [10/100], Loss: 17.7218
Epoch [11/100], Loss: 17.7239
Epoch [12/100], Loss: 17.7252
Epoch [13/100], Loss: 17.7256
Epoch [14/100], Loss: 17.7242
Epoch [15/100], Loss: 17.7260
Epoch [16/100], Loss: 17.7259
Epoch [17/100], Loss: 17.7282
Epoch [18/100], Loss: 17.7265
Epoch [19/100], Loss: 17.7269
Epoch [20/100], Loss: 17.7256
Epoch [21/100], Loss: 17.7273
Epoch [22/100], Loss: 17.7261
Epoch [23/100], Loss: 17.7246
Epoch [24/100], Loss: 17.7277
Epoch [25/100], Loss: 17.7278
Epoch [26/100], Loss: 17.7278
Epoch [27/100], Loss: 17.7271
Epoch [28/100], Loss: 17.7287
Epoch [29/100], Loss: 17.7267
Epoch [30/100], Loss: 17.7271
Epoch [31/100], Loss: 17.7267
Epoch [32/100], Loss: 17.7289
Epoch [33/100], Loss: 17.7267
Epoch [34/100], Lo